In [1]:
import face_recognition
from PIL import Image
import cv2
import dlib
import numpy as np
import os

In [2]:
#通过三个点计算夹角
#b为夹角位置所在的点
#默认θ=1计算弧度，θ!不等于1时计算角度
def cos_angle(a,b,c,θ = 1):
    x,y = b-a,b-c
    Lx = np.sqrt(x.dot(x))
    Ly = np.sqrt(y.dot(y))
    cos_angle = x.dot(y)/(Lx*Ly)
    # 根据条件选择是计算弧度还是角度
    if θ != 1:
        return np.arccos(cos_angle)*360/2/np.pi
    else:
        return np.arccos(cos_angle)


In [3]:
#利用人脸标记模型 shape_predictor_68_face_landmarks.dat 标记人脸节点
def trait_angle(path):
    detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor(
        r'shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks.dat')
    img = cv2.imread(path)
    faces = detector(img, 1)
    feas = []  # 关键点
    if (len(faces) > 0):
        for k, d in enumerate(faces):
            cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), (255, 255, 255))
            shape = landmark_predictor(img, d)

            for i in range(68):
                num = str(shape.part(i))[1:-1].split(",")
                feas.append([int(num[0]), int(num[1])])

    feas = np.array(feas)
    s_fa = feas[45, :][1] - feas[36, :][1]
    a, b, c = feas[45, :], feas[36, :], np.array(feas[45, :][0], feas[36, :][1])
    if abs(s_fa) > 5:
        if s_fa > 0 and cos_angle(a, b, c,θ=4) >35:
            return cos_angle(a, b, c,θ=4)
        elif s_fa < 0 and cos_angle(a, b, c,θ=4) >35:
            return 360-cos_angle(a, b, c, θ=4)
        else:
            return 0
    else:
        return 0

In [4]:
def normalization(input,output):
    path =input
    out_path = output

    # 读取图片并识别人脸
    img = face_recognition.load_image_file(path)
    face_locations = tuple(list(face_recognition.face_locations(img)[0]))

    # 重新确定切割位置并切割
    top = face_locations[0]
    right = face_locations[1]
    bottom = face_locations[2]
    left = face_locations[3]
    cutting_position = (left, top, right, bottom)
    # 切割出人脸
    im = Image.open(path)

    region = im.crop(cutting_position)

    # 人脸缩放
    a = 50  # 人脸方格大小
    if region.size[0] >= a or region.size[1] >= a:
        region.thumbnail((a, a), Image.ANTIALIAS)
    else:
        region = region.resize((a, a), Image.ANTIALIAS)
    # 人脸旋转
    θ =trait_angle(path)
    # region = region.rotate(θ)
    # 保存人脸
    region.save(out_path)

In [5]:
# 数据集地址  最后的斜杆一定不要漏
path = 'D:/QIN/Face-classfication/TestDatabase_1/'
# 处理后的数据集
path_normalization = 'D:/QIN/Face-classfication/TestDatabase_1/normalization/'
# 模型保存地址
model_path = './人脸识别/model.ckpt'
# tfrecord文件存放路径
TFRECORD_FILE = "./人脸识别/tfrecords/"

#全局one-hot编码空间
label_binarizer = ""

In [6]:
#测试预处理
normalization('D:/QIN/Face-classfication/TestDatabase_1/1_1.jpg','D:/QIN/Face-classfication/TestDatabase_1/normalization/11_1.jpg')

In [7]:
#读取文件批量处理
def read_img(path):
    map_path, map_relative = [path +x for x in os.listdir(path) if os.path.isfile(path + x) ], [y for y in os.listdir(path)]
    return map_path, map_relative

# 读取图片并处理保存
def read_dispose_img(path):
    map_path ,map_relative= read_img(path)
    print(map_path)
    print(map_relative)
    imgs = []
    labels = []
    for idx, folder, in enumerate(map_path):
        print("读取图并处理中......"+path_normalization+map_relative[idx])
        normalization(folder,path_normalization+map_relative[idx])

In [8]:
#     处理照片
read_dispose_img(path)

['D:/QIN/Face-classfication/TestDatabase_1/10_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/1_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/2_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/3_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/4_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/5_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/6_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/7_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/8_1.jpg', 'D:/QIN/Face-classfication/TestDatabase_1/9_1.jpg']
['10_1.jpg', '1_1.jpg', '2_1.jpg', '3_1.jpg', '4_1.jpg', '5_1.jpg', '6_1.jpg', '7_1.jpg', '8_1.jpg', '9_1.jpg', 'normalization']
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_1/normalization/10_1.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_1/normalization/1_1.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_1/normalization/2_1.jpg
读取图并处理中......D:/QIN/Face-classfication/TestDatabase_1/normalization/3_1.jpg
读取图并处理中......D:/QIN/Face-classfication/TestD

In [9]:
#默认按列归一化
def def_normalization(x,h = 1):
    # 记录归一化全局最大值最小值，回代时需要
    if h==1:
        #按列归一化
        amin, amax = np.min(x, 0), np.max(x, 0)
        xx = (x - amin) / (amax - amin)
    else:
        #按行归一化
        amin, amax = np.min(x, 1), np.max(x, 1)
        xx = ((x.T - amin) / (amax - amin)).T
    #记录归一化最大值最小值，回代时需要
    return xx

#使用one-hot编码，将离散特征的取值扩展到了欧式空间
def def_one_hot(x):
    if label_binarizer == "":
        binarizer = sklearn.preprocessing.LabelBinarizer()
    else:
        binarizer = label_binarizer
    binarizer.fit(range(max(x)+1))
    y= binarizer.transform(x)
    return y

In [10]:
# 读取处理后的图片，顺序打乱，划分测试和训练  ---》期中C 的文件夹内放置独一无二的类
def read_new_img(path):
    map_path, map_relative = read_img(path)
    imgs=[]
    labels=[]
    for idx, folder, in enumerate(map_path):
        img = io.imread(path_normalization+map_relative[idx])
        img = transform.resize(img, (50, 50))
        imgs.append(img)
        labels.append(int(map_relative[idx].split(",")[0].split("_")[0]))
    x_data, x_label = np.array(imgs), np.array(labels)
    print("labels:")
    print(labels)
    print("x_labels:")
    print(x_label)
    print("x_data:")
    print(x_data)
#     # 打乱顺序
#     num_example = data.shape[0]
#     arr = np.arange(num_example)
#     np.random.shuffle(arr)
#     data = data[arr]
#     label = label[arr]
#     # 将所有数据分为训练集和验证集
#     ratio = 0.8
#     s = np.int(num_example * ratio)
#     return data[:s], data[s:], def_one_hot(label[:s]), def_one_hot(label[s:])
    map_path, map_relative = read_img(path + 'c/')
    print(map_path)
    print(map_relative)
    imgs=[]
    labels=[]
    for idx, folder, in enumerate(map_path):
        img = io.imread(path_normalization + 'c/' +map_relative[idx])
        img = transform.resize(img, (50, 50))
        imgs.append(img)
        labels.append(int(map_relative[idx].split(",")[0].split("_")[0]))
    c_data, c_label = np.array(imgs), np.array(labels)
    x_data, c_data, def_one_hot(x_label), def_one_hot(c_label)
    return x_data, c_data, def_one_hot(x_label), def_one_hot(c_label)

In [11]:
#####网络层
#初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.01)#生成一个截断的正态分布
    return tf.Variable(initial,name=name)

#初始化偏置
def bias_variable(shape,name):
    initial = tf.constant(0.01,shape=shape)
    return tf.Variable(initial,name=name)

#卷积层
def conv2d(x,W):
    #x input tensor of shape `[batch, in_height, in_width, in_channels]`
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#depthwise
def tf_depthwise(input,weights ):
    depthwise=tf.nn.depthwise_conv2d( input, weights, [1, 1, 1, 1], padding='SAME' ) 
    return depthwise